In [4]:
!pip install aicrowd-cli

In [5]:

API_KEY = '00755d5c519897336fea1fa8d5274df4'
!aicrowd login --api-key $API_KEY


API Key valid
Saved API Key successfully!


In [6]:
!mkdir data
!aicrowd dataset download --challenge emotion-detection -j 3 -o data

mkdir: cannot create directory ‘data’: File exists
val.csv:   0% 0.00/262k [00:00<?, ?B/s]
val.csv: 100% 262k/262k [00:00<00:00, 1.03MB/s]

test.csv: 100% 642k/642k [00:00<00:00, 1.81MB/s]
train.csv: 100% 2.30M/2.30M [00:00<00:00, 4.42MB/s]


In [7]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import tensorflow_hub as hub

In [8]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
validation_data = pd.read_csv('data/val.csv')

In [9]:
x_train,y_train = train_data['text'].to_numpy(),train_data['label'].to_numpy()

In [10]:
x_val,y_val = validation_data['text'].to_numpy(),validation_data['label'].to_numpy()

In [11]:
use_mod = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',trainable = False,output_shape=(1,))

In [12]:
model = tf.keras.Sequential([
                             tf.keras.layers.InputLayer(input_shape=[],dtype = 'string'),
                             use_mod,
                             tf.keras.layers.Dense(64,activation = 'relu'),
                             tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [13]:
model.compile(loss = tf.keras.losses.binary_crossentropy,
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [14]:
x_train

array(['takes no time to copy/paste a press release', "You're delusional",
       "Jazz fan here. I completely feel. Lindsay Mann cousins has more votes than Lindsay Mann, and Lindsay Mann hasn't even stepped on the court this year",
       ..., 'Friendly fire dude, I wanted the other criminal ',
       'Yes, exactly. Fix a date and if he still procrastinates ditch him. Or remain his penpal for the rest of your life.',
       "Ferrets are such good ESA's though! Good for you at least, I love ferrets with all of my heart."],
      dtype=object)

In [15]:
history = model.fit(tf.expand_dims(x_train,axis = 1),y_train,
          epochs = 5, 
          validation_data = (x_val,y_val),
          verbose = 2)

Epoch 1/5
977/977 - 14s - loss: 0.4174 - accuracy: 0.8101 - val_loss: 0.3828 - val_accuracy: 0.8249
Epoch 2/5
977/977 - 9s - loss: 0.3819 - accuracy: 0.8247 - val_loss: 0.3777 - val_accuracy: 0.8246
Epoch 3/5
977/977 - 9s - loss: 0.3716 - accuracy: 0.8318 - val_loss: 0.3769 - val_accuracy: 0.8206
Epoch 4/5
977/977 - 9s - loss: 0.3607 - accuracy: 0.8380 - val_loss: 0.3736 - val_accuracy: 0.8252
Epoch 5/5
977/977 - 9s - loss: 0.3490 - accuracy: 0.8440 - val_loss: 0.3745 - val_accuracy: 0.8241


In [16]:
prediction = model.predict(test_data['text'])

In [17]:
test_data.head()

,text,label
0,I was already over the edge with Cassie Zamora...,0
1,I think you're right. She has oodles of cash a...,0
2,Haha I love this. I used to give mine phone bo...,1
3,Probably out of desperation as they going no a...,0
4,Sorry !! You’re real good at that!!,0


In [18]:
test_data.label = tf.cast(tf.round(tf.squeeze(prediction)),tf.int32)

In [19]:
test_data.head()

,text,label
0,I was already over the edge with Cassie Zamora...,1
1,I think you're right. She has oodles of cash a...,0
2,Haha I love this. I used to give mine phone bo...,0
3,Probably out of desperation as they going no a...,0
4,Sorry !! You’re real good at that!!,1


In [20]:
test_data.to_csv('submission.csv',index = False)

In [21]:
train_data = train_data.append(validation_data)

In [22]:
x_train,y_train = train_data['text'].to_numpy(),train_data['label'].to_numpy()

In [23]:
len(x_train)

34728

In [24]:
use_mod = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',trainable = False,output_shape=(1,))

In [25]:
model = tf.keras.Sequential([
                             tf.keras.layers.InputLayer(input_shape=[],dtype = 'string'),
                             use_mod,
                             tf.keras.layers.Dense(64,activation = 'relu'),
                             tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [26]:
model.compile(loss = tf.keras.losses.binary_crossentropy,
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [27]:
model_2_history = model.fit(x_train,y_train,epochs = 10 )

Epoch 1/10
1086/1086 [==============================] - 13s 10ms/step - loss: 0.4174 - accuracy: 0.8114
Epoch 2/10
1086/1086 [==============================] - 10s 10ms/step - loss: 0.3820 - accuracy: 0.8240
Epoch 3/10
1086/1086 [==============================] - 10s 10ms/step - loss: 0.3720 - accuracy: 0.8306
Epoch 4/10
1086/1086 [==============================] - 10s 9ms/step - loss: 0.3601 - accuracy: 0.8372
Epoch 5/10
1086/1086 [==============================] - 10s 10ms/step - loss: 0.3489 - accuracy: 0.8438
Epoch 6/10
1086/1086 [==============================] - 10s 10ms/step - loss: 0.3372 - accuracy: 0.8496
Epoch 7/10
1086/1086 [==============================] - 10s 10ms/step - loss: 0.3257 - accuracy: 0.8560
Epoch 8/10
1086/1086 [==============================] - 10s 9ms/step - loss: 0.3146 - accuracy: 0.8622
Epoch 9/10
1086/1086 [==============================] - 10s 10ms/step - loss: 0.3040 - accuracy: 0.8691
Epoch 10/10
1086/1086 [==============================] - 10s 10ms/

In [28]:
prediction = model.predict(test_data['text'])

In [29]:
test_df = pd.read_csv('data/test.csv')

In [30]:
test_df.label =  tf.cast(tf.round(tf.squeeze(prediction)),tf.int32)

In [31]:
test_df.head()

,text,label
0,I was already over the edge with Cassie Zamora...,1
1,I think you're right. She has oodles of cash a...,0
2,Haha I love this. I used to give mine phone bo...,0
3,Probably out of desperation as they going no a...,0
4,Sorry !! You’re real good at that!!,1


In [32]:
test_df.to_csv('another.csv',index = False)

In [33]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.5MB/s 
     |████████████████████████████████| 3.3MB 27.4MB/s 
     |████████████████████████████████| 901kB 34.6MB/s 


In [34]:
# # Import required packages
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

In [35]:
# !wget https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english/blob/main/tf_model.h5

In [36]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [37]:
# x = tokenizer('this is it')

In [38]:
# import torch

In [39]:
# inputs = tokenizer("Jazz fan here. I completely feel. Lindsay Mann cousins has more votes than Lindsay Mann, and Lindsay Mann hasn't even stepped on the court this year", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# outputs.logits.detach().numpy().argmax()

In [40]:
# len(test_data)

In [41]:
# pred = []
# for scentence in test_data['text'].to_numpy():
#   inputs = tokenizer(scentence, return_tensors="pt")
#   labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
#   outputs = model(**inputs, labels=labels)
#   pred.append(outputs.logits.detach().numpy().argmax())


In [42]:
# test_data['label']  = pred

In [43]:
# test_data.to_csv('hugging_face_sub.csv',index = False)

In [12]:
# inputs = tokenizer(list(test_data['text'][:1000]), return_tensors="pt",truncation = True, padding = True)
# outputs = model(**inputs)
# outputs.logits.detach().numpy().argmax(axis = 1)

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])